In [185]:
from lxml import html
from scipy import misc
import requests
from PIL import Image
import numpy as np
from io import BytesIO
import os


In [192]:
#lowercase, hyphen-delimited
def get_img_urls_by_artist(artist='claude-monet'):
    url = 'https://www.wikiart.org/en/'+artist+'/all-works/text-list'
    works_page = requests.get(url)
    tree = html.fromstring(works_page.content)
    painting_li = tree.xpath('//li[@class="painting-list-text-row"]')
    return list(map(lambda li: li[0].get('href')[3:], painting_li))
    

In [4]:
img_urls = get_img_urls_by_artist()

<Element html at 0x21f028759f8>


In [135]:
#unused
class StatusIndicator:
    def __init__(self, total):
        self.total = total
        self.n = 0
        self.info = 'test info'
        
    def progress(self, total):
        n=0
        while n < total:
            done = '#' * (n+1)
            todo = '-' * (total-n-1)
            s = '<{0}>'.format(done + todo)
            if not todo:
                s += '\n'        
            if n > 0:
                s = '\r' + s
            #s = self.info + '\r' + s
            print(s, end='')
            yield n
            n+=1
            
    def set_info(msg):
        this.info = msg

In [193]:
def load_wikiart_imgs(img_urls, size='Large'):
    base = 'https://uploads3.wikiart.org/images'
    success = 0
    failure = 0
    images = []
    for img_url in img_urls:
        painting_url = base + img_url
        extension = '.jpg!'+size+'.jpg'
        full_url = painting_url + extension

        res = requests.get(full_url)
        tl = tr = bl = br = np.zeros((256, 256, 3))
        if res.status_code == 404:
            full_url = painting_url + '(1)' + extension
            res = requests.get(full_url)
        if res.status_code == 200 and ('jpeg' in res.headers['content-type'] or 'jpg' in res.headers['content-type']):
            success += 1
            full_img = np.array(Image.open(BytesIO(res.content)))
            try:
                images.append(tl[:, :] + full_img[:256, :256])
                images.append(tr[:, :] + full_img[:256, -256:])
                images.append(bl[:, :] + full_img[-256:, :256])
                images.append(br[:, :] + full_img[-256:, -256:])
            except:
                print("Image too small: ", full_url)
        elif res.status_code == 404:
            failure += 1
            print('Unable to find image: ', full_url)
        else:
            failure += 1
            print('Unknown failure: ', full_url)
    return success, failure, np.array(images)

In [194]:
def download_wikiart_imgs(img_urls, path, size='Large'):
    base = 'https://uploads3.wikiart.org/images'
    success = 0
    failure = 0
    
    if not os.path.exists('my_folder'):
        os.makedirs(path)
    for img_url in img_urls:
        painting_url = base + img_url
        extension = '.jpg!'+size+'.jpg'
        full_url = painting_url + extension
        res = requests.get(full_url, stream=True)
        if res.status_code == 404:
            full_url = painting_url + '(1)' + extension
            res = requests.get(full_url)
        if res.status_code == 200 and ('jpeg' in res.headers['content-type'] or 'jpg' in res.headers['content-type']):
            success += 1
            with open(path+'/'+str(success+failure)+'.jpg', 'wb') as f:
                for chunk in res.iter_content(1024):
                    f.write(chunk)
        elif res.status_code == 404:
            failure += 1
            print('Unable to find image: ', full_url)
        else:
            failure += 1
            print('Unknown failure: ', full_url)
    return success, failure

In [187]:
#succ, fail, images = load_wikiart_imgs(img_urls)

In [195]:
img_urls = get_img_urls_by_artist(artist='eugene-boudin')

In [196]:
download_wikiart_imgs(img_urls, 'boudin-paintings')

Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/the-farm-in-saint-sim%C3%A9on-1856(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/forgiveness-in-the-st-anne-chapel-at-palud-plonevez-porzay-finistere-1858(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/study-of-fishing-boats(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/sky-1863(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/the-beach-at-deauville-1863(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/the-beach-at-trouville-1865(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/the-entrance-1865(1).jpg!Large.jpg
Unable to find image:  https://uploads3.wikiart.org/images/eugene-boudin/the-beach-at-trouville-1869(1).jpg!Large.jpg
Unable to find image:  https://upload

(532, 28)